In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
# %mpy -s {"/dev/cu.usbmodem_fs3_1"}
# %mpy -s {"/dev/cu.usbmodem2101"}
# %mpy -s {"/dev/cu.usbmodem1101"}
%mpy -s {"/dev/cu.usbmodem101"}

In [ ]:
!mpremote reset

In [ ]:
# %%micropython
import micropython
import time
from config import config
from machine import RTC, Timer
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.time import ptp
from mp_libs.time import time as mp_time

NUM_SYNC_CYCLES = 15
TIMEOUT_MSEC = 5000

rtc = RTC()
sync_timer = Timer(1)
start_sync = False
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport._transport

# Work-around: disabling the power-saving mode on the STA_IF interface
# epn.wifi._sta.config(pm=epn.wifi._sta.PM_NONE)

miniot_network.connect()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Initial channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")
print(f"peers: {epn.epn.get_peers()}")
print(f"Active?: {epn.wifi._sta.active()}")

# Perform espnow scan to set the right channel
miniot_network.scan(timeout=500)
print(f"Final channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

# Start ptp sync timer
def timer_cb(timer):
    global start_sync
    start_sync = True
sync_timer.init(period=15000, mode=Timer.PERIODIC, callback=timer_cb)

while True:
    rxed_packets = []
    data_available = False

    # Receive data
    try:
        data_available = miniot_network.receive(rxed_packets)
    except Exception:
        data_available = False

    if start_sync:
        start_sync = False
        print(f"{ptp.time_sync(is_async=False, rx_fxn=miniot_network.receive)}")
        print(f"{rtc.datetime()}")
        print(f"{mp_time.get_fmt_datetime()}")


In [ ]:
# %%micropython
import network
import espnow
import time

channel = 5

sta = network.WLAN(network.WLAN.IF_STA)
sta.active(True)
sta.config(channel=channel)
print(sta.config("channel"))
print(sta.config("mac"))

e = espnow.ESPNow()
e.active(True)

peer = b'p\x04\x1d\xad|\xc0'
e.add_peer(peer)
print(e.get_peers())

for i in range(15):
    print(f"Send attempt {i}")
    e.send(peer, f"Sending on channel {channel}")
    time.sleep_ms(750)

e.send(peer, b"end")

In [ ]:
!mpremote reset